# Utilisation de Bokeh pour des visualisations interactives

In [1]:
#import des bibliothèques nécessaires
from collections import OrderedDict
from math import log, sqrt

import numpy as np
import pandas as pd
from six.moves import cStringIO as StringIO

from bokeh.plotting import figure, show, output_file

In [2]:
resultats = """
Clients,                        Résultat 1, Résultat 2, Résultat 3, renouvellement
client 1,      10,        5,            2,        negative
client 2,       10,         8,          9,     negative
client 3,                3,          1,          1,      negative
client 4,           8.5,        1.2,          1,        negative
client 5,                1,          2,            2,     negative
client 6,          8.5,        2,            4,      positive
client 7,                10,        4,          1,      positive
client 8, 1,          4,          8,    positive
client 9,            8,        1,            1.6,      positive
client 10,               1,      1,         7,    positive
"""

result_color = OrderedDict([
    ("Résultat 1",   "#0d3362"),
    ("Résultat 2", "#c64737"),
    ("Résultat 3",     "black"  ),
])

renew_color = {
    "positive" : "#aeaeb8",
    "negative" : "#e69584",
}

df = pd.read_csv(StringIO(resultats),
                 skiprows=1,
                 skipinitialspace=True,
                 engine='python')

In [3]:
#on a créé les données
df

,Clients,Résultat 1,Résultat 2,Résultat 3,renouvellement
0,client 1,10.0,5.0,2.0,negative
1,client 2,10.0,8.0,9.0,negative
2,client 3,3.0,1.0,1.0,negative
3,client 4,8.5,1.2,1.0,negative
4,client 5,1.0,2.0,2.0,negative
5,client 6,8.5,2.0,4.0,positive
6,client 7,10.0,4.0,1.0,positive
7,client 8,1.0,4.0,8.0,positive
8,client 9,8.0,1.0,1.6,positive
9,client 10,1.0,1.0,7.0,positive


In [14]:
#on organise le graphique
width = 800
height = 800
inner_radius = 90
outer_radius = 300 - 10

minr = 0
maxr = 20
a = (outer_radius - inner_radius) / (minr - maxr)
b = inner_radius - a * maxr

def rad(mic):
    return a * mic + b

big_angle = 2.0 * np.pi / (len(df) + 1)
small_angle = big_angle / 7

p = figure(plot_width=width, plot_height=height, title="",
    x_axis_type=None, y_axis_type=None,
    x_range=(-420, 420), y_range=(-420, 420),
    min_border=0, outline_line_color="black",
    background_fill_color="#f0e1d2", border_fill_color="#f0e1d2",
    toolbar_sticky=False)

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# annular wedges
angles = np.pi/2 - big_angle/2 - df.index.to_series()*big_angle
colors = [renew_color[gram] for gram in df["renouvellement"]]
p.annular_wedge(
    0, 0, inner_radius, outer_radius, -big_angle+angles, angles, color=colors,
)

# small wedges
p.annular_wedge(0, 0, inner_radius, rad(df["Résultat 1"]),
                -big_angle+angles+5*small_angle, -big_angle+angles+6*small_angle,
                color=result_color["Résultat 1"])
p.annular_wedge(0, 0, inner_radius, rad(df["Résultat 2"]),
                -big_angle+angles+3*small_angle, -big_angle+angles+4*small_angle,
                color=result_color["Résultat 2"])
p.annular_wedge(0, 0, inner_radius, rad(df["Résultat 3"]),
                -big_angle+angles+1*small_angle, -big_angle+angles+2*small_angle,
                color=result_color["Résultat 3"])

# circular axes and lables
labels = np.arange(0, 10)
radii = a * labels  + b
p.circle(0, 0, radius=radii, fill_color=None, line_color="white")
p.text(0, radii[:-1], [str(r) for r in labels[:-1]],
       text_font_size="8pt", text_align="center", text_baseline="middle")

# radial axes
p.annular_wedge(0, 0, inner_radius-10, outer_radius+10,
                -big_angle+angles, -big_angle+angles, color="black")

# labels
xr = radii[0]*np.cos(np.array(-big_angle/2 + angles))
yr = radii[0]*np.sin(np.array(-big_angle/2 + angles))
label_angle=np.array(-big_angle/2+angles)
label_angle[label_angle < -np.pi/2] += np.pi # easier to read labels on the left side
p.text(xr, yr, df["Clients"], angle=label_angle,
       text_font_size="9pt", text_align="center", text_baseline="middle")

#on ajoute des légendes
p.circle([-40, -40], [-370, -390], color=list(renew_color.values()), radius=5)
p.text([-30, -30], [-370, -390], text=["Renouvellement-" + gr for gr in renew_color.keys()],
       text_font_size="7pt", text_align="left", text_baseline="middle")

p.rect([-40, -40, -40], [18, 0, -18], width=30, height=13,
       color=list(result_color.values()))
p.text([-15, -15, -15], [18, 0, -18], text=list(result_color),
       text_font_size="9pt", text_align="left", text_baseline="middle")

output_file("clients.html", title="Exemple clients Bokeh")

INFO:bokeh.core.state:Session output file 'clients.html' already exists, will be overwritten.


In [15]:
show(p)